In [13]:
pip install scikit-learn


     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.6 kB ? eta -:--:--
     -------------------------------------- 60.6/60.6 kB 811.9 kB/s eta 0:00:00
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
    --------------------------------------- 0.2/10.6 MB 4.1 MB/s eta 0:00:03
   - -------------------------------------- 0.4/10.6 MB 4.0 MB/s eta 0:00:03
   -- ------------------------------------- 0.6/10.6 MB 4.1 MB/s eta 0:00:03
   --- ------------------------------------ 1.0/10.6 MB 5.1 MB/s eta 0:00:02
   ---- ----------------------------------- 1.3/10.6 MB 5.4 MB/s eta 0:00:02
   ----- ---------------------------------- 1.5/10.6 MB 5.1 MB/s eta 0:00:02
   ------ --------------------------------- 1.6/10.6 MB 5.0 MB/s eta 0:00:02
   ------ --------------------------------- 1.7/10.6 MB 4.8 MB/s eta 0:00:02
   ------ --------------------------------- 1.7/10.6 MB 4.8 MB/s eta 0:00:02
   ------ --

In [14]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from collections import Counter
from IPython.display import display, Image
from sklearn.metrics import classification_report, confusion_matrix

In [16]:


# Initialize the data generator with augmentation options
datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize the images to [0, 1]
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    r'E:\XAMPP\htdocs\Frontend\Backend\Sapota\br_data',
    target_size=(224, 224),  # Depending on the model architecture
    batch_size=32,
    class_mode='categorical',
    subset="training"
)
validation_generator = datagen.flow_from_directory(
    r'E:\XAMPP\htdocs\Frontend\Backend\Sapota\br_data',
    target_size=(224, 224),  # Depending on the model architecture
    batch_size=32,
    class_mode='categorical',subset="validation"
)


Found 850 images belonging to 2 classes.
Found 212 images belonging to 2 classes.


In [17]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
import numpy as np 

# Load the base model
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
model = Model(inputs=base_model.input, outputs=x)

def extract_features(generator, sample_count):
    # Assuming the model's output is 1024-dimensional features
    # Initialize arrays to hold the extracted features and labels.
    # Note: We use None for flexible sizing in the features' first dimension.
    features = np.zeros((0, 1024))  # Adjust 1024 if a different model output size is expected
    labels = np.zeros((0,))
    
    # Keep track of the number of samples processed
    processed_samples = 0
    
    for inputs_batch, labels_batch in generator:
        # Ensure not to process more than the sample_count
        if processed_samples < sample_count:
            # Predict features for the current batch of images
            features_batch = model.predict(inputs_batch)
            actual_batch_size = features_batch.shape[0]
            
            # Update the total number of processed samples
            processed_samples += actual_batch_size
            
            # Append the predicted features and the true labels to their respective arrays
            features = np.append(features, features_batch, axis=0)
            labels = np.append(labels, np.argmax(labels_batch, axis=1), axis=0)
            
            # If we've processed enough samples, break from the loop
            if processed_samples >= sample_count:
                break
        else:
            break
    
    # If we've processed more samples than needed, truncate the arrays
    if processed_samples > sample_count:
        features = features[:sample_count]
        labels = labels[:sample_count]
    
    return features, labels

train_features, train_labels = extract_features(train_generator, 200) # Adjust 200 to actual size
validation_features, validation_labels = extract_features(validation_generator, 50) # Adjust 50 to actual size


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 791ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 736ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 799ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 637ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 648ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 657ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 727ms/step


In [18]:
model.save("feature_extractor.h5")

In [19]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix
from sklearn.preprocessing import StandardScaler
import joblib


# It's often a good idea to scale features before using SVM
scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features)
validation_features_scaled = scaler.transform(validation_features)

# Train the SVM model
svm_model = SVC(kernel='linear', C=1.0, decision_function_shape='ovo')
svm_model.fit(train_features_scaled, train_labels)

# Predict on the validation set
validation_predictions = svm_model.predict(validation_features_scaled)

# Evaluate the model
print("Validation Accuracy: ", accuracy_score(validation_labels, validation_predictions))
print("Classification Report:\n", classification_report(validation_labels, validation_predictions))


Validation Accuracy:  1.0
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        25
         1.0       1.00      1.00      1.00        25

    accuracy                           1.00        50
   macro avg       1.00      1.00      1.00        50
weighted avg       1.00      1.00      1.00        50



In [20]:
joblib.dump(scaler, 'scaler.save')

['scaler.save']

In [21]:
confusion_matrix(validation_labels, validation_predictions)

array([[25,  0],
       [ 0, 25]], dtype=int64)

In [22]:
import joblib
joblib.dump(svm_model, 'svm_model.pkl')

['svm_model.pkl']

In [29]:
from tensorflow.keras.models import load_model
import joblib
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.applications.mobilenet import preprocess_input
import numpy as np
def load_and_preprocess_image(image_path):
    # Load the image file, resizing it to 224x224 pixels (as expected by MobileNet)
    img = load_img(image_path, target_size=(224, 224))
    # Convert the image to a numpy array
    img_array = img_to_array(img)
    # Expand dimensions to match the shape expected by the pre-trained model: (1, 224, 224, 3)
    img_array_expanded = np.expand_dims(img_array, axis=0)
    # Preprocess the image for the pre-trained model
    return preprocess_input(img_array_expanded)

# Load the feature extraction model
feature_extraction_model_path = r'E:\XAMPP\htdocs\Frontend\Backend\Sapota\Bruise and no bruise\feature_extractor.h5'
feature_extraction_model = load_model(feature_extraction_model_path)

# Load the scaler
scaler_path =r'E:\XAMPP\htdocs\Frontend\Backend\Sapota\Bruise and no bruise\scaler.save'
scaler = joblib.load(scaler_path)

# Load the SVM classifier
svm_classifier_path = r'E:\XAMPP\htdocs\Frontend\Backend\Sapota\Bruise and no bruise\svm_model.pkl'
svm_classifier = joblib.load(svm_classifier_path)

# Assuming you have a function to load and preprocess images named `load_and_preprocess_image`
preprocessed_image = load_and_preprocess_image(r"E:\XAMPP\htdocs\Frontend\Backend\Sapota\br_data\no bruises\1132 (27).jpg")

# Extract features
features = feature_extraction_model.predict(preprocessed_image)

# Scale features
scaled_features = scaler.transform(features.reshape(1, -1))

# Predict with the SVM model
predicted_class = svm_classifier.predict(scaled_features)
print("Predicted class:", predicted_class)
if predicted_class==0:
    result="Bruises"
    print(result)
else:           
    result="No Bruises" 
    print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 710ms/step
Predicted class: [1.]
No Bruises
